In [2]:
import pandas as pd
data = pd.read_csv("train.csv")
# data = pd.read_csv("/content/drive/MyDrive/colab/hackfinal/train_part_small_8x.csv")
bundles_b = data[data.os.str.lower() == 'android'].bundle.unique()
ios_bundles = data[data.os.str.lower() == 'ios'].bundle.unique()

In [ ]:
!pip install google_play_scraper
!pip install async-itunes-app-scraper-dmi

In [7]:
from multiprocessing.pool import ThreadPool
from google_play_scraper import app

pool = ThreadPool(100)
def get_request(bundle):
    try:
        result = app(
            bundle,
            lang='ru',
            country='ru')
        if 'screenshots' in result and 'comments' in result:
            del result['screenshots']
            del result['comments']
        return (bundle, {'result': result})
    except Exception as e:
        return (bundle, {'error': str(e)})

import time, io, json
from tqdm.auto import tqdm
import json

import gc
t1 = time.time()
for idx, bundles_a in enumerate([bundles_b[i : i+1000] for i in range(0, len(bundles_b), 1000)]):
    gc.collect()
    bundle_info = pool.map(get_request, list(bundles_a))
    t2 = time.time()
    print(idx, t2 - t1)
    bundle_info = dict(bundle_info)
    with open('g_store_description_part{}.json'.format(idx), 'w') as f:
        json.dump(bundle_info, f, ensure_ascii=False)

0 3.1379313468933105


In [ ]:
import nest_asyncio
from itunes_app_scraper.scraper import AppStoreScraper
# from app_store_scraper import AppStore
# from pprint import pprint
import asyncio

nest_asyncio.apply()

class TickBatcher(object):

    def __init__(self):
        self.one_batch = {}
        self.scrapper = AppStoreScraper()

    async def sub(self, ids):
        try:
            async for item in self.scrapper.get_multiple_app_details(ids, country='ru', lang='ru'):
                self.one_batch[ids[0]] = item
        except: 
            print("error occured with {}".format(ids))

proc = TickBatcher()
loop = asyncio.get_event_loop()
batch_size = 1
import tqdm
for i in tqdm.tqdm(range(len(ios_bundles) // batch_size + 1)):
    batch = ios_bundles[i * batch_size: (i+1)*batch_size]

    loop.run_until_complete(proc.sub(batch))

loop.close()


In [24]:
with open("ios_apps_info.json", 'w') as f:
    json.dump(proc.one_batch, f)

In [ ]:
info_andr = {}
for i in range(73+1): # for case if 73 parts of json were saved
    filename = f"g_store_description_part_{i}.json"
    with open(filename, "r") as f:
        part_dict = json.load(f)
    info_andr.update(part_dict)

with open("ios_apps_info.json", "r") as f:
    info_ios = json.load(f)

print(len(info_andr), len(info_ios))
for dic in info_andr, info_ios:
    for k in list(dic.keys()):
        e = dic[k].get('error')
        if e is not None and e != '':
            del dic[k]
print(len(info_andr), len(info_ios))

for k in list(info_andr.keys()):
    info_andr[k] = info_andr[k]['result']
    info_andr[k]['os'] = 'android'

for k in info_ios:
    info_ios[k]['os'] = 'ios'

print(len(info_andr), len(info_ios))
for k in list(info_ios.keys()):
    kind = info_ios[k].get('kind')
    if kind != 'software':
        print('delete', k)
        del info_ios[k]
print(len(info_andr), len(info_ios))

for k in list(info_andr.keys()):
    v = info_andr[k]
    a = v.get('description')
    if a is None:
        print('delete', k)
        del info_andr[k]

for k in list(info_andr.keys()):
    a = info_andr[k]['description']
    info_andr[k]['adve'] = a
    del info_andr[k]['description']

for k in list(info_ios.keys()):
    kind = info_ios[k].get('kind')
    if kind != 'software':
        print('delete', k)
        del info_ios[k]

info = info_andr.copy()
info.update(info_ios)

import json
with open("/content/hackfinal/apps_info.json", 'w') as f:
    json.dump(info, f, ensure_ascii=False)